In [1]:
# Copyright (c) Microsoft Corporation. All rights reserved
# Licensed under the MIT License.
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
import os

import numpy as np

import fiona

from temporal_cluster_matching import utils, DataInterface, algorithms

## NAIP / Poultry barns

In [2]:
labeled_idxs, labeled_years = utils.get_poultry_barn_labels("../data/")
labeled_idxs = np.array(labeled_idxs)
labeled_years = np.array(labeled_years)
all_idxs, all_years, all_distances = utils.get_results("../results/kl/poultry_barns-64-200/results.csv")
all_idxs = np.array(all_idxs)
theta = 1.931520 # we get this estimate of theta from the poultry_barns-64-200 row in ../results/heuristic-theta_results.csv

distances, years = [], []
for idx in labeled_idxs:
    distances.append(all_distances[idx])
    years.append(all_years[idx])

predicted_years = utils.decision_function(distances, years, theta, 2016)
predicted_years = np.array(predicted_years)

print(utils.uncertain_accuracy(labeled_years, predicted_years))

0.938


In [3]:
all_predicted_years = utils.decision_function(all_distances, all_years, theta, 2016)

In [4]:
all_predicted_years = np.array(all_predicted_years)

In [5]:
years = list(range(2011, 2019))
idxs_per_year = []
for year in years:
    idxs_per_year.append(np.where(all_predicted_years <= year)[0])    

In [6]:
for idxs, year in zip(idxs_per_year, years):
    print(year, len(idxs))

2011 5099
2012 5126
2013 5363
2014 5377
2015 5655
2016 5683
2017 6013
2018 6013


In [7]:
with fiona.open("../data/Delmarva_PL_House_Final2_epsg26918.geojson", "r") as source:

    sink_schema = source.schema
    sink_schema["properties"]["first_year"] = "int"

    with fiona.open(
        "../predictions/poultry_barns-64-200_predictions.geojson",
        "w",
        crs=source.crs,
        driver="GeoJSON",
        schema=sink_schema,
    ) as sink:
        for i, row in enumerate(source):
            row["properties"]["first_year"] = int(all_predicted_years[i])
            sink.write(row)

In [8]:
for i, year in enumerate(years):
    with fiona.open("../data/Delmarva_PL_House_Final2_epsg26918.geojson", "r") as source:

        sink_schema = source.schema

        with fiona.open(
            "../predictions/poultry_barns-64-200_predictions_%d.geojson" % (year),
            "w",
            crs=source.crs,
            driver="GeoJSON",
            schema=sink_schema,
        ) as sink:
            for j, row in enumerate(source):
                if j in idxs_per_year[i]:
                    sink.write(row)